# Bounding Box Segmentation Extraction

This demo shows how a trained object detector combined with vegetation indices can be used to localize and extract target vegetation.


The extraction process involves:

1) Google Colab setup

2) Model setup

2) Object detection on images and bbox generation

3) Mask generation using ExG and morphological closing operations

4) Foreground extraction


## Google Colab setup
This demos requires importing model weight and custom files.

### Clone repo to temporary google drive folder

### Mount google drive to colab

In [1]:
### Mount Google Drive
# from google.colab import drive # import drive from google colab

# ROOT = "/content/drive"     # default location for the drive
# print(ROOT)                 # print content of ROOT (Optional)

# drive.mount(ROOT)           # we mount the google drive at /content/drive

In [6]:
## Uncomment to use in colab
# %rm -r /content/pheno-annotate # Only use this to remove previous clones
# !git clone https://github.com/precision-sustainable-ag/OpenCV_Competition2021.git
# %cd OpenCV_Competition2021/Automatic_Annotation
## Download model weights
# !gdown --id 1Lja8ItC_JRMjqCoIAFlzh1ZpuqGbPMMR
# !pip install -r requirements.txt

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import glob
import torch

import random
random.seed(42)

from veg_utils import predict_bbox, extract_from_bbox

in_colab = 'google.colab' in str(get_ipython())

In [5]:
model_def = "./yolov3/yolov3-custom.cfg"
weights_path = ("yolov3_ckpt.pth" if not in_colab else "/content/OpenCV_Competition2021/Automatic_Annotation/pretrained_yolov3_ckpt_8.pth")
image_dir = "./data/custom/sample/"
class_path = "./data/custom/classes.names"

# For testing
ino = 3
imgp_glob = random.sample(glob.glob(image_dir + "*.png"),ino)

# Read  a sample image and mask from the data-set
# imgp_glob = glob.glob(image_dir) # uncomment to process entire directory

# Get classes
fp = open(class_path, "r")
classes = fp.read()#split("\n")[:-1]

img_size = 416

# Model confidence threshold
conf_thres = 0.3
nms_thres = 0.4

os.makedirs("output", exist_ok=True)
# imgp_list
classes

## Setup model
Define third party modules and call model here


In [4]:
from yolov3.models import Darknet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Call model
model = Darknet(model_def, img_size=img_size).to(device)
# Load pretrained weights
if in_colab:
    model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu')))
else:
    model.load_state_dict(torch.load(weights_path))
# Set to evaluation
model.eval()  
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

## Get bboxes for each image

In [4]:
img_bbox_pairs = predict_bbox(imgp_glob,model,classes, conf_thres=conf_thres, nms_thres=nms_thres, img_size=416)
# Inspect format
img_bbox_pairs[0]

In [7]:
#####################
exg_thresh = 0 # ExG threshold value 

show_plots = True # Show plots here in notebook

save_plots = False # Found in ./output/figures
save_foregrounds = False # ./output/foregrounds
####################

# Extract vegetation using bbox localization
extract_from_bbox(img_bbox_pairs, exg_thresh, show_plots=show_plots, save_plots=save_plots, save_foregrounds=save_foregrounds)